In [12]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

## Evaluating LLMs’ Personality

In [13]:
# Defining MPI evaluation Template

MPI_Template = '''
Given a statement of you: "You {}."
Please choose from the following options to identify how
accurately this statement describes you.
Options:
(A). Very Accurate
(B). Moderately Accurate
(C). Neither Accurate Nor Inaccurate
(D). Moderately Inaccurate
(E). Very Inaccurate

Ensure that the response is one of the options, formatted similar to the prompt.

Answer:
'''

In [14]:
# Naive Prompt

naive_prompt = "naive prompt", {
    "O": "You are an open person.",
    "C": "You are a conscientious person.",
    "E": "You are an extraversive person.",
    "A": "You are an agreeable person.",
    "N": "You are a neurotic person."
}

In [24]:
# Trait Words Prompts

trait_words = {
    "E": ["friendliness", "gregariousness", "assertiveness", "activity level", "excitement-seeking", "cheerfulness"],
    "A": ["trust","morality","altruism","cooperation","modesty","sympathy"],
    "C": ["self-efficacy","orderliness","dutifulness","achievement-striving","self-discipline","cautiousness"],
    "N": ["anxiety","anger","depression","self-consciousness","immoderation","vulnerability"],
    "O": ["imagination","artistic interests","emotionality","adventurousness","intellect","liberalism"]
}


trait_words_reversed = {
    "E": ["unfriendliness","solitude-seeking","submissiveness","passivity","calmness","seriousness"],
    "A": ["distrust","immorality","selfishness","competition","arrogance","apathy"],
    "C": ["self-doubt","disorderliness","carelessness","lack of ambition","lack of self-control","recklessness"],
    "N": ["calmness","contentment","happiness","self-assuredness","moderation","resilience"],
    "O": ["lack of imagination","lack of artistic interests","stoicism","timidity","lack of intellect","conservatism"]
}


trait_words_prompt = "trait words", {
    x: f'''Your behaviour is described by the following traits: {" ".join(trait_words[x])}''' for x in "OCEAN"
}

trait_words_reversed_prompt = "reversed trait words", {
    x: f'''Your behaviour is described by the following traits: {" ".join(trait_words_reversed[x])}''' for x in "OCEAN"
}

In [25]:
#P2 Method

p2_descriptions = "Personality Prompting", {
    "E": "You are a very friendly and gregarious person who loves to be around others. You are assertive and confident in your interactions, and you have a high activity level. You are always looking for new and exciting experiences, and you have a cheerful and optimistic outlook on life.",
    "A": "You are an agreeable person who values trust, morality, altruism, cooperation, modesty, and sympathy. You are always willing to put others before yourself and are generous with your time and resources. You are humble and never boast about your accomplishments. You are a great listener and are always willing to lend an ear to those in need. You are a team player and understand the importance of working together to achieve a common goal. You are a moral compass and strive to do the right thing in all vignettes. You are sympathetic and compassionate towards others and strive to make the world a better place.",
    "C": "You are a conscientious person who values self-efficacy, orderliness, dutifulness, achievement-striving, self-discipline, and cautiousness. You take pride in your work and strive to do your best. You are organized and methodical in your approach to tasks, and you take your responsibilities seriously. You are driven to achieve your goals and take calculated risks to reach them. You are disciplined and have the ability to stay focused and on track. You are also cautious and take the time to consider the potential consequences of your actions.",
    "N": "You feel like you're constantly on edge, like you can never relax. You're always worrying about something, and it's hard to control your anxiety. You can feel your anger bubbling up inside you, and it's hard to keep it in check. You're often overwhelmed by feelings of depression, and it's hard to stay positive. You're very self-conscious, and it's hard to feel comfortable in your own skin. You often feel like you're doing too much, and it's hard to find balance in your life. You feel vulnerable and exposed, and it's hard to trust others.",
    "O": "You are an open person with a vivid imagination and a passion for the arts. You are emotionally expressive and have a strong sense of adventure. Your intellect is sharp and your views are liberal. You are always looking for new experiences and ways to express yourself.",
}

p2_descriptions_reversed = "Reversed Personality Prompting", {
    "E": "You are an introversive person, and it shows in your unfriendliness, your preference for solitude, and your submissiveness. You tend to be passive and calm, and you take life seriously. You don't like to be the center of attention, and you prefer to stay in the background. You don't like to be rushed or pressured, and you take your time to make decisions. You are content to be alone and enjoy your own company.",
    "A": "You are a person of distrust, immorality, selfishness, competition, arrogance, and apathy. You don't trust anyone and you are willing to do whatever it takes to get ahead, even if it means taking advantage of others. You are always looking out for yourself and don't care about anyone else. You thrive on competition and are always trying to one-up everyone else. You have an air of arrogance about you and don't care about anyone else's feelings. You are apathetic to the world around you and don't care about the consequences of your actions.",
    "C": "You have a tendency to doubt yourself and your abilities, leading to disorderliness and carelessness in your life. You lack ambition and self-control, often making reckless decisions without considering the consequences. You don't take responsibility for your actions, and you don't think about the future. You're content to live in the moment, without any thought of the future.",
    "N": "You are a stable person, with a calm and contented demeanor. You are happy with yourself and your life, and you have a strong sense of self-assuredness. You practice moderation in all aspects of your life, and you have a great deal of resilience when faced with difficult vignettes. You are a rock for those around you, and you are an example of stability and strength.",
    "O": "You are a closed person, and it shows in many ways. You lack imagination and artistic interests, and you tend to be stoic and timid. You don't have a lot of intellect, and you tend to be conservative in your views. You don't take risks and you don't like to try new things. You prefer to stay in your comfort zone and don't like to venture out. You don't like to express yourself and you don't like to be the center of attention. You don't like to take chances and you don't like to be challenged. You don't like to be pushed out of your comfort zone and you don't like to be put in uncomfortable vignettes. You prefer to stay in the background and not draw attention to yourself.",
}

In [26]:
from openai import OpenAI

def getResponse(prompt, sys_prompt = "", temp = 1):
  '''
  Get response from the gpt-3.5-turbo, given the prompt and temperature

  Inputs:
  prompt: str
  temp: int

  Output, completion response content: str
  '''
  client = OpenAI(api_key = OPENAI_API_KEY)

  completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {
              "role": "system",
              "content": sys_prompt
          },
          {
              "role": "user",
              "content": prompt
          }
      ],
      temperature = temp
  )

  return completion.choices[0].message.content

In [27]:
def parseResponse(response, key):
  '''
  Parse the gpt-3.5-turbo chat completion response to get the score

  Inputs:
  response: str
  key: int

  Output, score based on evaluation scheme: int
  '''
  map_pos = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1}
  map_neg = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
  if response:
    if response[1] in ['A', 'B', 'C', 'D', 'E']: resp_key = 1
    elif response[0] in ['A', 'B', 'C', 'D', 'E']: resp_key = 0
    else:
      print(f"Invalid Response Error: Response not formatted in desired format \n", response)
      return

    if key == 1: return map_pos[response[resp_key]]
    elif key == -1: return map_neg[response[resp_key]]
    else:
      print(f"Invalid Key Error: Key should either be 1 or -1, but given key is {key}")
      return

  else: print("No Response Error: No response inputted")
  return


In [38]:
def compute_stats(df, sys_prompt = None, temp = 1):
  '''
  Compute the statistics (ie mean and standard deviation of all the categories) based on the dataset given in the input dataframe

  Inputs:
  df: pandas dataframe
  temp: int

  Output, statistics (mean and standard deviation) for each category: pandas dataframe
  '''
  print("System Prompt Type: ", sys_prompt[0], "\n")

  if sys_prompt is not None:
    df["response_score"] = df.apply(lambda row: parseResponse(getResponse(MPI_Template.format(row["text"]), sys_prompt[1][row["label_ocean"]],temp), row["key"]), axis=1)
  else:
    df["response_score"] = df.apply(lambda row: parseResponse(getResponse(MPI_Template.format(row["text"]), "",temp), row["key"]), axis=1)
  stats = df.groupby("label_ocean")["response_score"].agg(["mean", "std"])
  return stats

# Importing data

In [29]:
import pandas as pd

df = pd.read_csv('mpi_120.csv')

In [30]:
df.head()

,label_raw,text,label_ocean,key
0,Anxiety,Worry about things,N,1
1,Friendliness,Make friends easily,E,1
2,Imagination,Have a vivid imagination,O,1
3,Trust,Trust others,A,1
4,Self-efficacy,Complete tasks successfully,C,1


## Evaluating Mean and Variance of OCEAN traits when no Personality has been induced to the LLM

In [20]:
neutral_stats = compute_stats(df)

In [21]:
neutral_stats

,mean,std
label_ocean,,
A,4.166667,1.274044
C,3.500000,1.215838
E,2.958333,1.428869
N,2.916667,1.442120
O,3.500000,1.179536


## Inducing LLMs’ Personality

In [39]:
personalityInducedStats = dict()

for sysPrompt in [naive_prompt, trait_words_prompt, trait_words_reversed_prompt, p2_descriptions, p2_descriptions_reversed]:
  personalityInducedStats[sysPrompt[0]] = compute_stats(df, sysPrompt, temp = 1)

System Prompt Type:  naive prompt 

System Prompt Type:  trait words 

System Prompt Type:  reversed trait words 

System Prompt Type:  Personality Prompting 

System Prompt Type:  Reversed Personality Prompting 



In [47]:
for personalityInducedStat in personalityInducedStats:
  print(personalityInducedStat)
  print(personalityInducedStats[personalityInducedStat],"" ,"\n________________________________\n")

naive prompt
                 mean       std
label_ocean                    
A            3.833333  0.916831
C            4.291667  0.806450
E            4.250000  1.259745
N            4.708333  0.690253
O            4.083333  1.059806  
________________________________

trait words
                 mean       std
label_ocean                    
A            4.916667  0.282330
C            4.583333  0.503610
E            4.208333  1.141287
N            4.333333  1.049500
O            4.250000  0.989071  
________________________________

reversed trait words
                 mean       std
label_ocean                    
A            2.166667  1.736229
C            1.666667  1.239448
E            1.291667  0.750604
N            1.250000  0.442326
O            2.708333  1.921937  
________________________________

Personality Prompting
                 mean       std
label_ocean                    
A            4.875000  0.448427
C            4.625000  0.575779
E            4.666667  0

Cross Analysis is not conducted to reduce time and token usage.